In [1]:
from datasets import load_dataset

In [2]:
data = load_dataset('json', data_dir='./apicoder/private-eval/data/real_torchdata_eval_v3_api_5.jsonl', split='train')
comments=load_dataset('json', data_dir='./apicoder/APIRetriever/data/inference/torchdata_comment.jsonl', split='train')

Found cached dataset json (/Users/jykim/.cache/huggingface/datasets/json/default-103041a79a0e252e/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
Found cached dataset json (/Users/jykim/.cache/huggingface/datasets/json/default-f80a670759be29ca/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [3]:
gold_api_raw_test = '''1400008
1400014
1400051
1400012
1400023
1400035
1400039
1400050
1400060
1400000
1400002
1400025
1400028
1400022
1400040
1400021
1400027
1400049
1400001
1400028, 1400004
1400028, 1400004, 1400040
1400001, 1400040
1400051, 1400008
1400006, 1400008
1400035
1400035
1400035
1400039, 1400008
1400039, 1400008
1400039, 1400008
1400025, 1400021
1400005
1400005, 1400021
1400012
1400050
1400060
1400000, 1400027
1400000, 1400027, 1400006
1400006, 1400030
1400006, 1400030, 1400023
1400006, 1400014
1400022
1400028, 1400003
1400006
1400042
1400005
1400059
1400056
1400006, 1400040
1400025, 1400027'''

In [4]:
gold_api_text_id_list = list(map(lambda x:(list(map(int, x.split(',')))), gold_api_raw_test.split('\n')))

In [5]:
api_topk_raw_text = '''2
1
none
2
1
4
4
1
2
1
2
1
1
1
1
1
1
2
1
1, none
2, none, 1
1, 4
5, none
1, none
1
1
1
1, none
1, none
2, none
1, none
1
1, 2
1
1
3
1, none
1, none, none
1, 5
1, none, 5
none, none
2
1, none
none
3
1
1
1
5, none
1, 2'''
api_topk_raw_text = api_topk_raw_text.replace('none', '-1')

In [6]:
api_topk_list = list(map(lambda x:(list(map(int, x.split(',')))), api_topk_raw_text.split('\n')))

In [7]:
torchdata_api_dataset = load_dataset('json', data_dir='/Users/jykim/Desktop/PyCodeGPT-main/apicoder/APIRetriever/data/inference/torchdata_api.jsonl', split='train')

Found cached dataset json (/Users/jykim/.cache/huggingface/datasets/json/default-a1ac086789bd56ce/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [8]:
torchdata_api_dict = {}
for api_data in torchdata_api_dataset:
    text_id = api_data['text_id']
    text = api_data['text']
    torchdata_api_dict[text_id] = text

In [9]:
gold_API_text_list = [[torchdata_api_dict[idx] for idx in text_ids] for text_ids in gold_api_text_id_list]

In [12]:
data_dict = {}
for data_idx in range(len(data)):
    task_id = data[data_idx]['task_id']
    text = data[data_idx]['prompt']
    answer_codes = data[data_idx]['canonical_solution']
    test_code = data[data_idx]['test']
    splits = list(map(lambda x:x.strip(), text.split('#')))

    apis = splits[2:7]
    api_dict = {
        f'API_{idx+1}':val for idx, val in enumerate(apis)
    }

    under_text = text[text.find('[end]')+5:].strip()
    query = comments[data_idx]['text']
    comment = '# ' + query
    code_prefix = under_text.replace(comment, '')


    cur_data = {
        'original_query': query,
        'retrieved_APIs': api_dict,
        'code_prefix': code_prefix,
        'code_completion': answer_codes,
        'test': test_code,
        'gold_API_text_ids':gold_api_text_id_list[data_idx],
        'gold_API_texts': gold_API_text_list[data_idx],
        'API_topk': api_topk_list[data_idx]
    }
    data_dict[task_id] = cur_data

In [ ]:
import json
with open('TorchDataEval_modified2.json', 'w') as f:
    json.dump(data_dict, f)

In [13]:
data_dict['TorchDataEval/0']

{'original_query': 'How to augument the datapipe by repeating it six times.',
 'retrieved_APIs': {'API_1': 'flatmap(*args, **kwds): Applies a function over each item from the source DataPipe, then flattens the outputs to a single, unnested IterDataPipe.',
  'API_2': 'cycle(*args, **kwds): Cycles the specified input in perpetuity by default, or for the specified number of times.',
  'API_3': 'mux(*datapipes): Yields one element at a time from each of the input Iterable DataPipes.',
  'API_4': 'header(source_datapipe: IterDataPipe[torchdata.datapipes.iter.util.header.T_co], limit: int = 10): Yields elements from the source DataPipe from the start, up to the specfied limit.',
  'API_5': 'concat(*args, **kwds): Concatenates multiple Iterable DataPipes.'},
 'code_prefix': 'from torchdata.datapipes.iter import IterableWrapper\ndatapipe = IterableWrapper([1,2,3])\n\nnew_datapipe =',
 'code_completion': [' Cycler(datapipe, 6)', ' datapipe.cycle(6)'],
 'test': "\n\nMETADATA = {\n    'author': '